<a href="https://colab.research.google.com/github/pcbzmani/Learning-repository-for-scala-and-pyspark/blob/main/datediff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pwd

'/content'

In [ ]:
ls

account_date.csv  sample_data/


In [ ]:
pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 35.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=dcf9cef733aba3e9ada01b6aa615318ba4761dabaa6313c2d9d0e9e6871e9222
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("datediff")\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [ ]:
df = spark.read\
        .format('csv')\
        .option('sep','|')\
        .option('inferSchema',True)\
        .option('header',True)\
        .load('/content/account_date.csv')

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col,datediff,row_number,lit,when,first,lag
from pyspark.sql.types import IntegerType

In [ ]:
w_up_cr = Window\
       .partitionBy('account_number')\
       .orderBy('date')\
       .rangeBetween(Window.unboundedPreceding, Window.currentRow)

In [ ]:
w_op_cr = Window\
            .partitionBy('account_number')\
            .orderBy('date')\
            .rangeBetween(-1,Window.currentRow)

In [ ]:
df = df.withColumn('first_date',first(df['date']).over(w_uf))\
       .withColumn('previous_date',first(df['date']).over(w_up_cr))

In [ ]:
df.show()

+--------------+----------+----------+-------------+
|account_number|      date|first_date|previous_date|
+--------------+----------+----------+-------------+
|           123|2016-01-01|2016-01-01|   2016-01-01|
|           123|2017-01-01|2016-01-01|   2016-01-01|
|           123|2018-01-01|2016-01-01|   2016-01-01|
|           123|2020-01-01|2016-01-01|   2016-01-01|
|           456|2019-01-01|2019-01-01|   2019-01-01|
|           456|2021-01-01|2019-01-01|   2019-01-01|
|           896|2021-01-01|2021-01-01|   2021-01-01|
+--------------+----------+----------+-------------+



In [ ]:
window_dt = Window\
            .partitionBy('Account_number')\
            .orderBy('Date')

In [ ]:
df = df.withColumn('first_dt_der',row_number().over(window_dt))

In [ ]:
df.show()

+--------------+----------+----------+-------------+------------+
|account_number|      date|first_date|previous_date|first_dt_der|
+--------------+----------+----------+-------------+------------+
|           123|2016-01-01|2016-01-01|   2016-01-01|           1|
|           123|2017-01-01|2016-01-01|   2016-01-01|           2|
|           123|2018-01-01|2016-01-01|   2016-01-01|           3|
|           123|2020-01-01|2016-01-01|   2016-01-01|           4|
|           456|2019-01-01|2019-01-01|   2019-01-01|           1|
|           456|2021-01-01|2019-01-01|   2019-01-01|           2|
|           896|2021-01-01|2021-01-01|   2021-01-01|           1|
+--------------+----------+----------+-------------+------------+



In [ ]:
df = df.withColumn('date_diff_first',datediff('date','first_date'))\
.withColumn('date_diff_previous',datediff('date','previous_date'))\
.withColumn('first_dt',when(df['first_dt_der'] == '1',lit('1'))\
            .otherwise(lit('0')))

In [ ]:
df.show()

+--------------+----------+----------+-------------+------------+---------------+------------------+--------+
|account_number|      date|first_date|previous_date|first_dt_der|date_diff_first|date_diff_previous|first_dt|
+--------------+----------+----------+-------------+------------+---------------+------------------+--------+
|           123|2016-01-01|2016-01-01|   2016-01-01|           1|              0|                 0|       1|
|           123|2017-01-01|2016-01-01|   2016-01-01|           2|            366|               366|       0|
|           123|2018-01-01|2016-01-01|   2016-01-01|           3|            731|               731|       0|
|           123|2020-01-01|2016-01-01|   2016-01-01|           4|           1461|              1461|       0|
|           456|2019-01-01|2019-01-01|   2019-01-01|           1|              0|                 0|       1|
|           456|2021-01-01|2019-01-01|   2019-01-01|           2|            731|               731|       0|
|         

In [ ]:
df = df.select('account_number','date','date_diff_first','date_diff_previous','first_dt')

In [ ]:
df.show()

+--------------+----------+---------------+------------------+--------+
|account_number|      date|date_diff_first|date_diff_previous|first_dt|
+--------------+----------+---------------+------------------+--------+
|           123|2016-01-01|              0|                 0|       1|
|           123|2017-01-01|            366|               366|       0|
|           123|2018-01-01|            731|               731|       0|
|           123|2020-01-01|           1461|              1461|       0|
|           456|2019-01-01|              0|                 0|       1|
|           456|2021-01-01|            731|               731|       0|
|           896|2021-01-01|              0|                 0|       1|
+--------------+----------+---------------+------------------+--------+



In [ ]:
df = df.withColumn('lag_prev1',lag(df['Date'],offset=1,default='2021-01-01').over(window_dt))

In [ ]:
df.show()

+--------------+----------+----------+-------------+--------+------------+------------------------+---------------------------+--------+----------+----------+
|Account_number|      Date|first_date|previous_date|constant|first_dt_der|month_between_date_first|month_between_date_previous|first_dt|  lag_prev| lag_prev1|
+--------------+----------+----------+-------------+--------+------------+------------------------+---------------------------+--------+----------+----------+
|           123|2016-01-01|2016-01-01|   2016-01-01|       1|           1|                       0|                          0|       1|      null|2021-01-01|
|           123|2017-01-01|2016-01-01|   2016-01-01|       1|           2|                      12|                         12|       0|2016-01-01|2016-01-01|
|           123|2018-01-01|2016-01-01|   2017-01-01|       1|           3|                      24|                         12|       0|2017-01-01|2017-01-01|
|           123|2020-01-01|2016-01-01|   2018-